In [1]:
import os
import re
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
sns.set('paper','whitegrid', font_scale=2, rc={"lines.linewidth": 2, 'grid.linestyle': '--'})

In [2]:
def get_summary_perfomance(local_dir: str, expt_name: str = None):
    summary_performance = {}
    expt_names = glob.glob(os.path.join(local_dir, "*_*"))
    for expt_name in expt_names:
        splits = expt_name.split("\\")[-1].split("__")
        if len(splits) == 3:
            algo, env_params, timelog = splits
        else:
            algo, timelog = splits
            env_params = "default"
        trials = glob.glob(os.path.join(expt_name, "ExperimentCV_*"))
        summary_trials = {}
        for trial in trials:
            with open(os.path.join(trial, "params.json"), 'r') as f:
                params = json.load(f)
            train_start = datetime.strptime(params["_train_start"], "%Y-%m-%d") + relativedelta(years=params["__trial_index__"])
            eval_start = datetime.strptime(params["_train_start"], "%Y-%m-%d") + relativedelta(years=params["__trial_index__"] + params["_train_years"])
            column_name = f"{train_start.year}(+{params['_train_years']}) | {eval_start.year}(+{params['_eval_years']})"
            
            try:
                perfomance_train = pd.read_csv(os.path.join(trial, "best-stats-train", "performance.csv"), index_col=0).rename(columns={'0': column_name})
                perfomance_eval = pd.read_csv(os.path.join(trial, "best-stats-eval", "performance.csv"), index_col=0).rename(columns={'0': column_name})
                summary_trials[column_name] = perfomance_eval
            except Exception as e:
                pass
        
        summary_performance[f"{algo}_{env_params}"] = summary_trials

    return summary_performance

In [3]:
# summary_perfomance = get_summary_perfomance("./ray_results/compare-action-3")
summary_perfomance = get_summary_perfomance("./ray_results/features_selection_10")

In [4]:
performance_expt = pd.DataFrame()
for expt_name, summary_trials in summary_perfomance.items():
    performance_cv = pd.DataFrame()
    for trial, performance in summary_trials.items():
        performance = performance.iloc[-4:, :].astype(float)
        performance_cv = pd.concat([performance_cv, performance], axis=1)


    performance_cv_mean = performance_cv.mean(axis=1)
    performance_cv_mean.name = expt_name
    performance_expt = pd.concat([performance_expt, performance_cv_mean], axis=1)
    
performance_expt

,A2C_default,DQN_default,PPO_default,SAC_default
Return (Ann.) [%],13.272513,27.123381,3.731098,19.873487
Volatility (Ann.) [%],12.439139,12.708330,12.233924,12.561945
Sharpe Ratio,1.072616,1.919754,0.400986,1.645851
Max. Drawdown [%],-10.978208,-6.188126,-9.588222,-8.221787
